In [1]:
import networkx as nx
import random
import math
import csv
import multiprocessing as mp
f = open("textfilter3.csv", "rb")
g = nx.read_edgelist(f)
num_edges = g.number_of_edges()
num_nodes = g.number_of_nodes()
f.close()

newgraph = nx.DiGraph()
newgraph.add_edges_from(g.edges(), positive="True") 
nx.write_edgelist(newgraph, "newgraph3.txt", data=['positive'])

num_sample_edges = int(math.floor(0.1 * num_edges))  # positive sample data
print(num_sample_edges)
sample_edges = random.sample(newgraph.edges(), num_sample_edges)
sample_g = nx.Graph()
sample_g.add_edges_from(sample_edges, positive = "True")
nx.write_edgelist(sample_g, "positivesample3.txt", data=['positive'])  

132482


In [2]:
fake_negative = open("negativesample3.txt","w")  # make negative sample the same as num_sample_edges(positive sample data) 
# and add it to sample_g, thus sample_g has data amount of 2*num_sample_edges(2 × 132482 records of edges)
i = 0
while i < num_sample_edges:  #132482 records
    edge = random.sample(sample_g.nodes(), 2)
    try:
        shortest_path = nx.shortest_path_length(sample_g,source=edge[0],target=edge[1])
        if shortest_path !=1:
            sample_g.add_edge(edge[0],edge[1], positive="False")   #add neagtive case            
            i += 1
    except:
        sample_g.add_edge(edge[0],edge[1], positive="False") 
        i +=1   
    fake_negative.write(str(edge[0])+" "+str(edge[1])+" False\n")
fake_negative.close()

In [3]:
print(sample_g.number_of_edges())
nx.write_edgelist(sample_g, "allsample3.txt", delimiter=',' , data = ['positive'])

264964


In [4]:
import networkx as nx
import math

def CommonNeighbors(u, v, g):
    try:
        u_neighbors = set(g.neighbors(u))
        v_neighbors = set(g.neighbors(v))
        return len(u_neighbors.intersection(v_neighbors))
    except:
        return 0

def AdamicAdar(u, v, g):
    try:
        u_neighbors = set(g.neighbors(u))
        v_neighbors = set(g.neighbors(v))
        aa = 0
        for i in u_neighbors.intersection(v_neighbors):
            try:
                aa += 1 / math.log(len(list(g.neighbors(i))))
            except:
                aa += 0
    except:
        aa = 0
    return aa

def ResourceAllocation(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    ra = 0
    for i in u_neighbors.intersection(v_neighbors):
        ra += 1 / float(len(list(g.neighbors(i))))
    return ra

def JaccardCoefficent(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    return len(u_neighbors.intersection(v_neighbors)) / float(len(u_neighbors.union(v_neighbors)))

def PreferentialAttachment(u, v, g):
    try:
        return len(list(g.neighbors(u)))*len(list(g.neighbors(v)))
    except:
        return 0


In [66]:
CommonNeighbors("235938", "4586866",g)

439

In [5]:
from pandas import DataFrame, Series
import pandas as pd
pd.read_table('allsample3.txt', sep = ',')
pdtable = pd.read_table('allsample3.txt', sep = ',', names = ["source", "target", "positive"])
print(pdtable[:2])
print(CommonNeighbors('2184483', '3348689',g))
print(type(pdtable["positive"]))
print(type(pdtable["target"][0]))

    source   target  positive
0  2789436  2203730      True
1  2789436  1467962      True
1
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>


In [6]:
pdtable.target = pdtable.target.astype(str)
pdtable.source = pdtable.source.astype(str)

In [7]:
pdtable["Common"] = pdtable.apply(lambda x: CommonNeighbors(x["source"], x["target"], g), axis = 1)  # x is the pd, apply feature functions
pdtable["Jaccard"] = pdtable.apply(lambda x: JaccardCoefficent(x["source"], x["target"], g), axis = 1)
pdtable["PreferentialAttachment"] = pdtable.apply(lambda x: PreferentialAttachment(x["source"], x["target"], g), axis = 1)
pdtable["AdamicAdar"] = pdtable.apply(lambda x: AdamicAdar(x["source"], x["target"], g), axis = 1)
pdtable["ResourceAllocation"] = pdtable.apply(lambda x: ResourceAllocation(x["source"], x["target"], g), axis = 1)
pdtable[:4]

,source,target,positive,Common,Jaccard,PreferentialAttachment,AdamicAdar,ResourceAllocation
0,2789436,2203730,True,43,0.112565,36036,7.807700,0.202805
1,2789436,1467962,True,54,0.113924,67760,10.368729,0.354982
2,2789436,812874,True,27,0.076705,21868,4.776423,0.141276
3,2789436,3183013,True,25,0.075988,14168,5.283508,0.285230


In [8]:
pdtable.to_csv('trainingdataframe.txt', sep='\t', header = ['source', 'target', 'positive','Common', 'Jaccard', "PreferentialAttachment", "AdamicAdar", "ResourceAllocation" ])

In [46]:
import csv
import multiprocessing as mp
f = open("testdata.csv", "rb")
df = pd.read_table('testdata.csv', sep = '\t', names = ["source", "target"])
df[:2]

,source,target
0,2184483,1300190
1,3151356,1452193
2,1579396,193159
3,1406432,2481036
4,2389638,593017
5,228206,212805
6,1237964,879115
7,3318124,1840575
8,4522929,1552625
9,3406737,3781412
